## Librerías

In [1]:
!pip install torch_geometric -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00


In [2]:
import networkx as nx
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Batch, Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, TransformerConv
from torch_geometric.utils import from_networkx

## Lectura desde Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/grafo_santiago_filtrado_con_embeddings.gexf"
Gx = nx.read_gexf(path)

# Graph Transformer

In [5]:
class GraphormerEncoder(nn.Module):
    def __init__(self, in_dim, hidden_dim=128, num_layers=4, num_heads=4, out_dim=128):
        super().__init__()
        # Proyección lineal del embedding de Alpha Earth
        self.input_proj = nn.Linear(in_dim, hidden_dim)

        # Capas del Graphormer
        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            self.layers.append(
                TransformerConv(
                    in_channels=hidden_dim,
                    out_channels=hidden_dim,
                    heads=num_heads,
                    concat=False,
                )
            )

        # Normalizaciones por capa (una LayerNorm por cada TransformerConv)
        self.norms = nn.ModuleList([nn.LayerNorm(hidden_dim) for _ in range(num_layers)])

        # Proyección final del embedding del grafo al espacio de salida (embedding de ciudad)
        self.out_proj = nn.Linear(hidden_dim, out_dim)

    def forward(self, x, edge_index, batch):
        """
        x: [N, in_dim]     - features de nodos
        edge_index: [2, E] - aristas del grafo en formato COO
        batch: [N]         - asigna cada nodo a un grafo (aquí, subgrafos / ciudades)
        """
        # Pasamos las features de los nodos al espacio oculto
        h = self.input_proj(x) # [N, hidden_dim]

        # Aplicamos num_layers veces: TransformerConv + residual + LayerNorm + ReLU
        for conv, norm in zip(self.layers, self.norms):
            h_res = h                      # conexión residual
            h = conv(h, edge_index)        # mensaje + atención entre nodos
            h = norm(h + h_res)            # residual + normalización
            h = F.relu(h)                  # no linealidad

        # Readout: agregamos todos los nodos de cada grafo en un solo vector (mean pooling)
        g = global_mean_pool(h, batch)

        # Proyección al espacio de embedding final
        g = self.out_proj(g)

        # Normalizamos para que los embeddings queden en la esfera unitaria (útil para contraste)
        g = F.normalize(g, p=2, dim=-1)
        return g

## Augmentaciones

In [6]:
# Devuelve un grafo que tiene drop_prob de sus aristas enmascaradas
def random_edge_dropout(data, drop_prob=0.2):
    edge_index = data.edge_index
    num_edges = edge_index.size(1)

    mask = torch.rand(num_edges, device=edge_index.device) > drop_prob
    new_edge_index = edge_index[:, mask]

    new_data = data.clone()
    new_data.edge_index = new_edge_index
    return new_data

# Devuelve un grafo que tiene mask_prob de sus features enmascaradas
def random_feature_masking(data, mask_prob=0.2):
    x = data.x.clone()
    mask = torch.rand_like(x) < mask_prob
    x[mask] = 0.0

    new_data = data.clone()
    new_data.x = x
    return new_data

# Aplica las dos transformaciones anteriores.
def graphcl_augment(data):
    v1 = random_edge_dropout(data, drop_prob=0.2)
    v1 = random_feature_masking(v1, mask_prob=0.2)

    v2 = random_edge_dropout(data, drop_prob=0.2)
    v2 = random_feature_masking(v2, mask_prob=0.2)

    return v1, v2

## Contrastive Loss

In [7]:
def nt_xent_loss(z1, z2, temperature=0.2):
    # Número de grafos en z1 (mismos que z2)
    batch_size = z1.size(0)

    # Concatenamos las 2 augmentaciones
    z = torch.cat([z1, z2], dim=0)

    # Similitud mediante producto punto
    sim = torch.matmul(z, z.t())

    # Escalamos con temperatura. Temperaturas más bajas hacen el contraste más agresivo
    sim = sim / temperature

    # Eliminamos la diagonal
    mask = torch.eye(2 * batch_size, device=z.device, dtype=torch.bool)
    sim = sim.masked_fill(mask, -1e9)

    # Construcción de labels
    # Las vistas augmentadas son vistas como pares positivos (i + batch_size)
    labels = torch.arange(2 * batch_size, device=z.device)
    labels = (labels + batch_size) % (2 * batch_size)

    # Cross-entropy
    loss = F.cross_entropy(sim, labels)
    return loss

## Procesamiento del dataset y obtención de subgrafos

In [8]:
# Se convierte al formato de torch_geometric
data_raw_full = from_networkx(Gx)

# Keys de los embeddings
A_keys = [f"A{i:02d}" for i in range(64)]

# Matriz de embeddings
feat_tensors_full = []
for k in A_keys:
    t = getattr(data_raw_full, k).view(-1, 1).float()
    feat_tensors_full.append(t)

# x_full tendrá shape [num_nodes, 64] con los embeddings AlphaEarth concatenados
x_full = torch.cat(feat_tensors_full, dim=1)

# Armamos el objeto Data de PyG para el grafo completo de Santiago
data_santiago = Data(
    x=x_full,
    edge_index=data_raw_full.edge_index,
    num_nodes=data_raw_full.num_nodes
)

# Extraemos los IDs de nodos y sus coordenadas geográficas desde el grafo de NetworkX
node_ids = list(Gx.nodes())
lats = np.array([Gx.nodes[n]['lat'] for n in node_ids])
lons = np.array([Gx.nodes[n]['lon'] for n in node_ids])

# Definimos la grilla en latitud/longitud:
num_lat_bins = 8
num_lon_bins = 8
min_nodes = 50   # para saltar subgrafos muy chicos

# Bordes de las celdas de la grilla
lat_bins = np.linspace(lats.min(), lats.max(), num_lat_bins + 1)
lon_bins = np.linspace(lons.min(), lons.max(), num_lon_bins + 1)

subgraphs_pyg = []

def build_data_from_nx(G_sub):
    """
    Convierte un subgrafo de NetworkX en un objeto Data de PyG,
    usando únicamente las 64 dimensiones A00..A63 como features de nodo.
    """
    data_raw = from_networkx(G_sub)

    feat_tensors = []
    for k in A_keys:
        t = getattr(data_raw, k).view(-1, 1).float()
        feat_tensors.append(t)
    x = torch.cat(feat_tensors, dim=1)

    data = Data(
        x=x,
        edge_index=data_raw.edge_index,
        num_nodes=data_raw.num_nodes
    )
    return data

for i in range(num_lat_bins):
    for j in range(num_lon_bins):
        mask = (
            (lats >= lat_bins[i]) & (lats < lat_bins[i + 1]) &
            (lons >= lon_bins[j]) & (lons < lon_bins[j + 1])
        )
        idxs = np.where(mask)[0]
        if len(idxs) < min_nodes:
            continue

        nodes_bin = [node_ids[k] for k in idxs]
        G_sub = Gx.subgraph(nodes_bin).copy()
        if G_sub.number_of_edges() == 0:
            continue

        data_sub = build_data_from_nx(G_sub)
        subgraphs_pyg.append(data_sub)

print("Subgrafos creados:", len(subgraphs_pyg))

Subgrafos creados: 46


## Definición del Dataset (pytorch)

In [9]:
class CityGraphDataset(Dataset):
    def __init__(self, graphs):
        super().__init__()
        self.graphs = graphs

    def len(self):
        return len(self.graphs)

    def get(self, idx):
        return self.graphs[idx]

dataset = CityGraphDataset(subgraphs_pyg)
loader = DataLoader(dataset, batch_size=8, shuffle=True)

## Bucle de entrenamiento

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dimension del embedding del nodo (64)
in_dim = dataset[0].x.size(1)
model = GraphormerEncoder(in_dim=in_dim, hidden_dim=128, num_layers=4, num_heads=4, out_dim=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, 51):
    model.train()
    total_loss = 0.0
    total_graphs = 0

    for batch_data in loader:
        batch_data = batch_data.to(device)

        # Separar en lista de grafos individuales
        data_list = batch_data.to_data_list()

        # Se crean las augmentaciones
        v1_list, v2_list = [], []
        for g in data_list:
            a1, a2 = graphcl_augment(g)
            v1_list.append(a1)
            v2_list.append(a2)

        # Se modelan como batches independientes
        v1_batch = Batch.from_data_list(v1_list).to(device)
        v2_batch = Batch.from_data_list(v2_list).to(device)

        # Se obtienen sus embeddings
        z1 = model(v1_batch.x, v1_batch.edge_index, v1_batch.batch)
        z2 = model(v2_batch.x, v2_batch.edge_index, v2_batch.batch)

        # Se cálcula la pérdida
        loss = nt_xent_loss(z1, z2, temperature=0.2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * z1.size(0)
        total_graphs += z1.size(0)

    avg_loss = total_loss / total_graphs
    print(f"Epoch {epoch:03d} | Loss: {avg_loss:.4f}")

Epoch 001 | Loss: 1.9272
Epoch 002 | Loss: 0.9886
Epoch 003 | Loss: 0.9316
Epoch 004 | Loss: 1.0652
Epoch 005 | Loss: 0.9961
Epoch 006 | Loss: 0.7809
Epoch 007 | Loss: 0.8608
Epoch 008 | Loss: 0.8196
Epoch 009 | Loss: 0.8723
Epoch 010 | Loss: 0.6315
Epoch 011 | Loss: 0.7257
Epoch 012 | Loss: 0.7251
Epoch 013 | Loss: 0.7509
Epoch 014 | Loss: 0.6153
Epoch 015 | Loss: 0.6322
Epoch 016 | Loss: 0.3792
Epoch 017 | Loss: 0.6779
Epoch 018 | Loss: 0.8287
Epoch 019 | Loss: 0.6364
Epoch 020 | Loss: 0.5621
Epoch 021 | Loss: 0.5944
Epoch 022 | Loss: 0.6309
Epoch 023 | Loss: 0.4832
Epoch 024 | Loss: 0.5493
Epoch 025 | Loss: 0.5223
Epoch 026 | Loss: 0.4246
Epoch 027 | Loss: 0.6704
Epoch 028 | Loss: 0.4387
Epoch 029 | Loss: 0.4975
Epoch 030 | Loss: 0.4613
Epoch 031 | Loss: 0.3391
Epoch 032 | Loss: 0.5603
Epoch 033 | Loss: 0.4406
Epoch 034 | Loss: 0.4940
Epoch 035 | Loss: 0.5353
Epoch 036 | Loss: 0.4311
Epoch 037 | Loss: 0.3060
Epoch 038 | Loss: 0.3617
Epoch 039 | Loss: 0.3930
Epoch 040 | Loss: 0.3971


## Sacamos el embedding de Santiago

In [23]:
data_santiago = data_santiago.to(device)
batch_full = torch.zeros(data_santiago.num_nodes, dtype=torch.long, device=device)

model.eval()
with torch.no_grad():
    emb_santiago = model(data_santiago.x, data_santiago.edge_index, batch_full)  # [1, out_dim]
    emb_santiago = emb_santiago[0]
    print("Embedding de Santiago:", emb_santiago.shape)

Embedding de Santiago: torch.Size([128])


## Verificar que acerca augmentaciones y separa subgrafos distintos

### Subgrafos al azar

In [25]:
model.eval()

# Tomamos un subgrafo al azar
g = random.choice(subgraphs_pyg)
g = g.to(device)

# Sacamos embedding del subgrafo original
b_orig = Batch.from_data_list([g]).to(device)
with torch.no_grad():
    z_orig = model(b_orig.x, b_orig.edge_index, b_orig.batch)[0]

# Augmentamos el grafo
a1, a2 = graphcl_augment(g)

b1 = Batch.from_data_list([a1]).to(device)
b2 = Batch.from_data_list([a2]).to(device)

with torch.no_grad():
    z1 = model(b1.x, b1.edge_index, b1.batch)[0]
    z2 = model(b2.x, b2.edge_index, b2.batch)[0]

# Cosenos entre original y augmentaciones
cos_orig_a1 = F.cosine_similarity(z_orig, z1, dim=0).item()
cos_orig_a2 = F.cosine_similarity(z_orig, z2, dim=0).item()
cos_a1_a2   = F.cosine_similarity(z1, z2, dim=0).item()

print("cosine(original, vista1) =", cos_orig_a1)
print("cosine(original, vista2) =", cos_orig_a2)
print("cosine(vista1, vista2)   =", cos_a1_a2)

# Sacamos otro grafo al azar
g2 = random.choice([h for h in subgraphs_pyg if h is not g]).to(device)
b3 = Batch.from_data_list([g2]).to(device)

with torch.no_grad():
    z3 = model(b3.x, b3.edge_index, b3.batch)[0]

# Comparamos su similaridad
cos_diff = F.cosine_similarity(z_orig, z3, dim=0).item()
print("cosine(original, otro subgrafo) =", cos_diff)


cosine(original, vista1) = 0.9545512199401855
cosine(original, vista2) = 0.9691970348358154
cosine(vista1, vista2)   = 0.995003342628479
cosine(original, otro subgrafo) = -0.06205920875072479


### Grafo Santiago

In [26]:
model.eval()

# grafo completo
g = data_santiago.to(device)

# augmentaciones del grafo completo
a1, a2 = graphcl_augment(g)

b_orig = Batch.from_data_list([g]).to(device)
b1     = Batch.from_data_list([a1]).to(device)
b2     = Batch.from_data_list([a2]).to(device)

with torch.no_grad():
    z_orig = model(b_orig.x, b_orig.edge_index, b_orig.batch)[0]
    z1     = model(b1.x, b1.edge_index, b1.batch)[0]
    z2     = model(b2.x, b2.edge_index, b2.batch)[0]

print("cos(original, vista1) =", F.cosine_similarity(z_orig, z1, dim=0).item())
print("cos(original, vista2) =", F.cosine_similarity(z_orig, z2, dim=0).item())
print("cos(vista1, vista2)   =", F.cosine_similarity(z1, z2, dim=0).item())

cos(original, vista1) = 0.9369595646858215
cos(original, vista2) = 0.9363678693771362
cos(vista1, vista2)   = 0.9998849630355835


In [27]:
g = random.choice(subgraphs_pyg).to(device)
b = Batch.from_data_list([g]).to(device)
with torch.no_grad():
    z_sub = model(b.x, b.edge_index, b.batch)[0]

cos_orig_sub = F.cosine_similarity(z_orig, z_sub, dim=0).item()
print("cos(Santiago completo, subgrafo al azar) =", cos_orig_sub)

cos(Santiago completo, subgrafo al azar) = 0.08978444337844849
